# Variational Autoencoders (VAEs)

Os Variational Autoencoders (VAEs) representam uma classe fundamental de modelos generativos profundos que combinam inferência bayesiana aproximada com redes neurais. Diferentemente dos Autoencoders (AEs) tradicionais, que mapeiam a entrada para um vetor latente fixo, os VAEs mapeiam a entrada para uma distribuição de probabilidade no espaço latente. Isso impõe uma estrutura contínua e regularizada ao espaço latente, permitindo não apenas a reconstrução eficaz dos dados, mas também a geração de novas amostras através da amostragem dessa distribuição aprendida.

## Formulação

O objetivo central em um modelo generativo é aprender a distribuição verdadeira dos dados $p_{data}(x)$ ou aproximá-la através de um modelo parametrizado $p_\theta(x)$. Em modelos de variáveis latentes, assumimos que os dados observáveis $x$ são gerados por um processo oculto envolvendo variáveis latentes $z$. A probabilidade marginal dos dados é dada pela integral:

$$p_\theta(x) = \int p_\theta(x|z) p(z) dz$$

Onde:
* $p(z)$ é a distribuição a priori (prior) das variáveis latentes, geralmente assumida como uma Gaussiana normal padrão $\mathcal{N}(0, I)$.
* $p_\theta(x|z)$ é a distribuição de verossimilhança (likelihood), modelada pelo decodificador (rede neural).

O cálculo direto desta integral é intratável para redes neurais profundas, pois requer a integração sobre todas as configurações possíveis de $z$. Da mesma forma, a distribuição posterior verdadeira $p_\theta(z|x) = p_\theta(x|z)p(z) / p_\theta(x)$ é intratável devido ao denominador.

## Inferência Variacional e ELBO

Para contornar a intratabilidade, introduzimos uma distribuição variacional $q_\phi(z|x)$ (o codificador) para aproximar a posterior verdadeira $p_\theta(z|x)$. Nosso objetivo é minimizar a Divergência de Kullback-Leibler (KL) entre a aproximação e a posterior verdadeira.

Entretanto, como não conhecemos a posterior verdadeira, maximizamos o limite inferior variacional, conhecido como **Evidence Lower Bound (ELBO)**:

$$\mathcal{L}(\theta, \phi; x) = \mathbb{E}_{q_\phi(z|x)}[\log p_\theta(x|z)] - D_{KL}(q_\phi(z|x) || p(z))$$

Esta função objetivo possui dois componentes cruciais:
1.  **Erro de Reconstrução**: $\mathbb{E}_{q_\phi(z|x)}[\log p_\theta(x|z)]$, que incentiva o decodificador a reconstruir os dados $x$ eficientemente a partir das amostras latentes.
2.  **Termo de Regularização**: $D_{KL}(q_\phi(z|x) || p(z))$, que força a distribuição latente aprendida a se aproximar da prior $p(z)$ (geralmente Gaussiana), garantindo continuidade e completude no espaço latente.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Local hyperparameters for data loading
batch_size = 128

# Data transformation
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Dataset setup
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## Arquitetura e o Truque de Reparametrização

A implementação do VAE exige um mecanismo diferenciável para amostrar $z$. O **Truque de Reparametrização** permite isso ao expressar a variável aleatória $z$ como uma transformação determinística de uma variável de ruído $\epsilon$:

$$z = \mu + \sigma \odot \epsilon, \quad \epsilon \sim \mathcal{N}(0, I)$$

Dessa forma, o gradiente pode fluir através de $\mu$ e $\sigma$ durante a retropropagação, enquanto a estocasticidade permanece isolada em $\epsilon$. A rede codificadora projeta a entrada em dois vetores: $\mu$ (média) e $\log(\sigma^2)$ (log-variância). O uso do logaritmo garante estabilidade numérica e evita restrições de não-negatividade na saída da rede.

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=400, latent_dim=20):
        super(VAE, self).__init__()
        
        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        
        # Decoder
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)
        
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc_mu(h1), self.fc_logvar(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return mu + eps * std
        else:
            return mu

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

## Função de Custo

A perda total é a soma do erro de reconstrução (Binary Cross Entropy) e da Divergência KL analítica para gaussianas.

$$D_{KL} = -\frac{1}{2} \sum_{j=1}^{J} (1 + \log(\sigma_j^2) - \mu_j^2 - \sigma_j^2)$$

In [ ]:
def loss_function(recon_x, x, mu, logvar):
    # Reconstruction term
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # KL Divergence term
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [ ]:
latent_dim = 2
model = VAE(latent_dim=latent_dim).to(device)

## Treinamento

Instanciamos o modelo e o otimizador localmente. Definimos uma dimensão latente de 2 (`latent_dim = 2`) especificamente para facilitar a visualização posterior do plano 2D, embora em aplicações reais de compressão dimensões maiores sejam preferíveis.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 15

model.train()
for epoch in range(1, epochs + 1):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f'Epoch: {epoch} \tAverage Loss: {train_loss / len(train_loader.dataset):.4f}')

## Análise de Reconstrução

O VAE deve ser capaz de mapear uma entrada $x$ para uma distribuição latente e, a partir de uma amostra $z$, recuperar uma aproximação $\hat{x}$ que preserve a semântica original.

Diferentemente de um Autoencoder determinístico, esperamos que as reconstruções do VAE sejam ligeiramente "enevoadas" (blurry). Isso ocorre devido à natureza probabilística do modelo e ao termo de regularização KL na função de perda, que força o espaço latente a ser suave e contínuo, sacrificando parte da fidelidade de alta frequência em prol de uma melhor capacidade de generalização e geração.

In [ ]:
num_imgs = 8
model.eval()
data, _ = next(iter(test_loader))
data = data.to(device)

with torch.no_grad():
    recon_batch, _, _ = model(data)

data = data.cpu()
recon_batch = recon_batch.cpu()

fig, axes = plt.subplots(2, num_imgs, figsize=(12, 4))

for i in range(num_imgs):
    # Original images
    axes[0, i].imshow(data[i].reshape(28, 28), cmap='gray')
    axes[0, i].axis('off')
    
    # Reconstructed images
    axes[1, i].imshow(recon_batch[i].reshape(28, 28), cmap='gray')
    axes[1, i].axis('off')

axes[0, 0].set_title("Original Input", fontsize=12, loc='left')
axes[1, 0].set_title("VAE Reconstruction", fontsize=12, loc='left')
plt.tight_layout()
plt.show()

## Geração Estocástica de Amostras

Uma vez treinado, o decodificador do VAE atua como um gerador determinístico $p_\theta(x|z)$. Para criar novas amostras que não existem no dataset original, realizamos a amostragem de vetores latentes a partir da distribuição a priori $p(z) = \mathcal{N}(0, I)$.

Como o treinamento forçou a distribuição latente dos dados a se aproximar dessa normal padrão (via termo KL), temos alta confiança de que pontos amostrados aleatoriamente dessa região resultarão em imagens semanticamente válidas.

In [ ]:
num_samples = 8
model.eval()
with torch.no_grad():
    z = torch.randn(num_samples, latent_dim).to(device)
    samples = model.decode(z).cpu()
    
plt.figure(figsize=(14, 4))
for i in range(num_samples):
    ax = plt.subplot(2, 8, i + 1)
    plt.imshow(samples[i].view(28, 28), cmap='gray')
    plt.axis('off')
    
plt.tight_layout()
plt.suptitle("Generated Samples from N(0, I)")
plt.show()

## Visualização do Manifold Latente

Como restringimos `latent_dim = 2`, podemos mapear diretamente o espaço latente 2D para o espaço de imagens. Ao invés de amostragem aleatória, criamos um *grid* de coordenadas linearmente espaçadas (usando a função de distribuição acumulada inversa para cobrir as regiões de maior densidade de probabilidade).

Isso nos permite visualizar a suavidade do espaço latente, observando como o modelo realiza a "metamorfose" contínua entre diferentes classes de dígitos ao percorrer os eixos $z_1$ e $z_2$.

In [ ]:
n = 20
digit_size = 28
model.eval()
figure = np.zeros((digit_size * n, digit_size * n))

grid_x = np.linspace(-1.5, 1.5, n)
grid_y = np.linspace(-1.5, 1.5, n)

with torch.no_grad():
    for i, yi in enumerate(grid_x):
        for j, xi in enumerate(grid_y):
            z_sample = torch.tensor([[xi, yi]]).float().to(device)
            x_decoded = model.decode(z_sample)
            digit = x_decoded.view(digit_size, digit_size).cpu().numpy()
            
            figure[i * digit_size: (i + 1) * digit_size,
                    j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.axis('off')
plt.title("Latent Space Manifold (2D)")
plt.show()